Housing Costs + Property Taxes over 5-year span (2018-2022)

In [5]:
import requests
import pandas as pd
import csv

def process_census_data(file_path):
    census_data_dict = {}
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)
        for row in csv_reader:
            if len(row) == 2:
                census_data_dict[row[0]] = row[1]
    return census_data_dict

def process_year_data(year):
    file_path = f'ACSST5Y{year}.S2506-Data.csv'
    meta_data_file_path = 'ACSST5Y2022.S2506-Column-Metadata.csv'

    df = pd.read_csv(file_path)
    census_data_dict = process_census_data(meta_data_file_path)

    df = df.rename(columns=census_data_dict)

    filtered_columns = [
        col for col in df.columns
        if "MONTHLY HOUSING COSTS" in col or "REAL ESTATE TAXES" in col or "Geography" in col or "Geographic Area Name" in col
    ]

    df = df[filtered_columns]
    df = df.iloc[1:]
    df['Geographic Area Name'] = df['Geographic Area Name'].str.replace('ZCTA5', '', regex=False).str.strip()

    # Add a year column
    df['Year'] = year

    return df

# Process data for years 2018 to 2022
years = range(2018, 2023)
dfs = [process_year_data(year) for year in years]

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined dataframe
print(combined_df.head())

# combined_df.to_csv('combined_housing_data_2018_2022.csv', index=False)

# Print some information about the combined dataset
# print(f"\nTotal number of rows: {len(combined_df)}")
print(f"Years included: {combined_df['Year'].unique()}")
# print(f"\nColumns in the dataset:")
# for col in combined_df.columns:
#     print(col)

<ipython-input-5-f4fcb94d0c96>:19: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-5-f4fcb94d0c96>:19: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128

        Geography Geographic Area Name  \
0  8600000US00601                00601   
1  8600000US00602                00602   
2  8600000US00603                00603   
3  8600000US00606                00606   
4  8600000US00610                00610   

  Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!Less than $200  \
0                                                 59                                                                                            
1                                                117                                                                                            
2                                                112                                                                                            
3                                                 54                                                                                            
4                     

In [17]:
combined_df.shape

(166908, 151)

Data Quality checks

In [21]:
combined_df.isna().sum().sum() / (combined_df.shape[0] * combined_df.shape[1])

0.010512989112295198

In [22]:
# Calculate total number of NaNs in the DataFrame
total_nan = combined_df.isna().sum().sum()

# print(f"Total number of NaNs in the DataFrame: {total_nan}")

# Calculate percentage of NaNs in each column
nan_percentage = combined_df.isna().mean() * 100

# print("\nPercentage of NaNs in each column:")
# for column, percentage in nan_percentage.items():
    # print(f"{column}: {percentage:.2f}%")

nan_percentage_sorted = nan_percentage.sort_values(ascending=False)

print("\nColumns sorted by NaN percentage (descending):")
for column, percentage in nan_percentage_sorted.items():
    print(f"{column}: {percentage:.2f}%")


Columns sorted by NaN percentage (descending):
Margin of Error!!Percent owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars): 39.69%
Estimate!!Percent owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars): 39.69%
Margin of Error!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars): 39.69%
Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars): 39.69%
Margin of Error!!Percent owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than $20,000!!30 percent or more: 0.00%
Estimate!!Percent owner-occupied housing units with a mortgage!!Owner-occupied housing units wi